In [130]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable
import os

In [131]:
check_gpu = torch.cuda.is_available()
check_gpu

True

In [132]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [133]:
check_devices = torch.cuda.get_device_name()
check_devices

'NVIDIA GeForce RTX 2080 Ti'

In [134]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [135]:
price_dataset = ohlc_df.close[-1000:]
time_index = ohlc_df.index[-1000:]
print(time_index[:5])
price_dataset.head()


DatetimeIndex(['2022-06-19 14:14:00', '2022-06-19 14:16:00',
               '2022-06-19 14:17:00', '2022-06-19 14:18:00',
               '2022-06-19 14:19:00'],
              dtype='datetime64[ns]', name='time', freq=None)


time
2022-06-19 14:14:00    3.7809
2022-06-19 14:16:00    3.7974
2022-06-19 14:17:00    3.8011
2022-06-19 14:18:00    3.8081
2022-06-19 14:19:00    3.8141
Name: close, dtype: float64

In [136]:
price_dataset_x = price_dataset[200:]
time_x = time_index[200:]
price_dataset_y = price_dataset[200:400]
time_y = time_index[200:400]
print(len(price_dataset_y))

200


In [137]:
price_test_x = price_dataset[400:600]
time_test_x = time_index[400:600]
price_test_y = price_dataset[600:900]
time_test_y = time_index[600:900]

In [138]:
seq_len = 1
batch_size = 64

In [139]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=5):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        #print(len(self.y))
        X = self.X[index:index+self.seq_len]
        try:
            y = self.y[index+self.seq_len]
        except:
            y = self.y[self.seq_len]
        return X, y

In [140]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=seq_len)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=seq_len)

In [141]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    drop_last = True, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    drop_last=True,
    shuffle=False)

In [142]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, input_dim , hidden_size , num_layers, batch_size):
        super(LSTM_Model, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_dim
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(input_size=input_dim , hidden_size = hidden_size , num_layers= num_layers )
        self.fc = torch.nn.Linear(hidden_size,1)

    def forward(self,x,hn,cn):
        out , (hn,cn) = self.lstm(x , (hn,cn))
        final_out = self.fc(out[-1])
        return final_out,hn,cn

    def predict(self,x):
        hn,cn  = self.init()
        final_out = self.fc(out[-1])
        return final_out

    def init(self):
        h0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        c0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        return h0 , c0


#device = torch.device('cpu')
#print(device)

input_dim = 1
hidden_size = seq_len
num_layers = 3
crypto_price_model = LSTM_Model(input_dim, hidden_size, num_layers, batch_size)


crypto_price_model#.to(device)

loss_function = torch.nn.MSELoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.017) # used adaptive moment estimation to optimize the model

In [143]:
#training loop
def train(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.train()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(1,batch_size,-1),hn,cn)
        out = out.view(-1)
        # print(out.shape)
        # print(y.shape)
        loss = loss_function(out.reshape(batch_size) , y)
        hn = hn.detach()
        cn = cn.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"train loss: {loss:>7f} ")
        loss_list.append(loss)
        predictions.append(out.detach().numpy())
        return predictions, loss_list
        

In [144]:
def test(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.eval()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(1,batch_size,1),hn,cn)
        loss = loss_function(out.reshape(batch_size) , y)
       
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"test loss: {loss:>7f} ")
        predictions.append(out.detach().numpy())
        loss_list.append(loss)
        return predictions, loss_list

In [200]:
epochs = 2
for epoch in range(epochs):
    print(f"epoch {epoch} ")
    train(train_loader, crypto_price_model)
    test_predictions = test(test_loader, crypto_price_model)
print(test_predictions[:][:][:][0])
fp = [num for num in test_predictions[0][0]]

epoch 0 
epoch 1 
[array([[2.7120526],
       [2.7120528],
       [2.7120533],
       [2.7120538],
       [2.7120528],
       [2.7120533],
       [2.7120523],
       [2.7120519],
       [2.71205  ],
       [2.7120495],
       [2.7120495],
       [2.7120464],
       [2.7120464],
       [2.712047 ],
       [2.7120457],
       [2.7120454],
       [2.7120466],
       [2.7120461],
       [2.7120466],
       [2.712045 ],
       [2.7120454],
       [2.712046 ],
       [2.7120447],
       [2.7120442],
       [2.7120438],
       [2.7120438],
       [2.7120433],
       [2.7120423],
       [2.7120433],
       [2.7120438],
       [2.7120433],
       [2.712043 ],
       [2.7120433],
       [2.7120433],
       [2.7120435],
       [2.7120438],
       [2.7120433],
       [2.7120433],
       [2.712043 ],
       [2.7120419],
       [2.7120423],
       [2.712042 ],
       [2.7120419],
       [2.7120419],
       [2.7120419],
       [2.7120414],
       [2.7120419],
       [2.7120428],
       [2.7120404],
 

In [201]:
print(len(test_predictions[0]))
print()
fig = go.Figure()
pred_line = go.Scatter(x=time_test_y[:len(fp)], y=fp, mode = 'lines', name = 'Predicted Price')
fig.add_trace(pred_line)

fig.add_trace(go.Scatter(x=time_test_y[:len(fp)], y=price_test_y[:len(fp)], mode = 'lines', name = 'Actual Price'))

fig.update_layout(xaxis_title="Time", yaxis_title="Actual Price ($)", title=f"Predicted Price vs Actual Price for '{crypto_name}'")

fig.show()

1



In [ ]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))

RuntimeError: CUDA error: device-side assert triggered